In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate
from pathlib import Path
from glob2 import glob
from sklearn.metrics import confusion_matrix
from fastai.callbacks import *
import torch.nn as nn
from fastai import *
from fastai.vision import *
import fastai
import pandas as pd
import numpy as np
import os
import zipfile as zf
import shutil
import re
import seaborn as sns

In [ ]:
path = Path(os.getcwd())/"mergeDataset"
path

In [ ]:
#load dataset
np.random.seed(42)
data = ImageDataBunch.from_folder(path,test="test",
                                  ds_tfms=get_transforms(),bs=512, size=(48,48)).normalize(imagenet_stats)

In [ ]:
#create model
learn = cnn_learner(data, models.resnet34,metrics=[accuracy])

In [ ]:
#finding learning rate
learn.unfreeze() # must be done before calling lr_find
learn.lr_find(start_lr=1e-6,end_lr=1e1)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(30, max_lr=5e-4,callbacks=[SaveModelCallback(learn, every='improvement', monitor='accuracy', name='best'),ShowGraph(learn),EarlyStoppingCallback(learn2 ,min_delta=0.01,monitor='valid_loss', patience=3)])

In [ ]:
#for confusion matrix
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()